# Generate Adversarial Nets

![gan1](https://user-images.githubusercontent.com/43749571/78065491-55e8d180-73ce-11ea-9b55-96b0018cc0e6.png)

## Abstract

GAN은 adversarial(적대적인) 과정을 통해 생성모델을 평가하는 framework 입니다.    
**minimax two-player game**

GAN은 동시에 두 개의 모델을 학습합니다. 
1. **Generate model (G)**: data 분포를 학습
2. **Discriminate model (D)**: trainining data로부터 나온 sample에서 확률을 추정 


* G는 D가 실수할 확률을 최대화 하고자 합니다. 
* G와 D는 multilayer perceptron으로 이루어진 구조이며, backpropagation으로 학습됩니다. 
* Markov Chain 같은 복잡한 수식 과정이 필요하지 않습니다. 

## Introduction

기존의 discriminative model과 관련된 Deep Learning에서의 성공들은  
- high dimentional, rich sensory data들이 class label을 구분하는 input으로 사용되었고,   
- well-behaved gradient를 갖는 linear units 을 사용한 
- backpropagation과 dropout 알고리즘 기반이었습니다. 


Deep generative model의 성공은 어려웠는데, 
- 복잡한 통계 계산 (likelihood 등)과 
- generative context에 linear model을 적용하기가 힘들다는 점이 그 이유였습니다. 

.


이 논문에서는, 위의 어려움을 해결하는 새로운 generative model estimation procedure을 제안합니다. 

**adversarial net framework**       
**discriminative model**은 sample이 model distribution으로부터 왔는지 (=G로부터 생성되었는지),    
data distribution으로부터 왔는지(=training data로 부터 왔는지) 학습합니다. 

< 비유 >      
**generative model**: 위조지폐 만드는 사람 
- fake currency를 만들고 -> use it without detection이 목표!
- 위조지폐를 진짜 지폐와 최대한 비슷하게 만들어서 경찰에게 걸리지 않으려고 합니다.   


**discriminative model**: 경찰 
- trying to detect the counterfeit currency 
- 가짜 지폐를 구별하려고 합니다. 


이를 계속 반복하면, 두 팀 각자의 성능은 모두 향상됩니다.  
결국에는 진짜로부터 가짜가 구별 되지 않을 때 까지 (경찰이 진짜와 가짜를 구별할 수 없을 때 까지) 진행됩니다. 

이 논문에서는, training algorithm과 optimization에 주목해 다음과 같은 상황들을 살펴보게 됩니다.   


**adversarial nets**     
1. generative model: multilayer perceptron을 통과시킨 랜덤 noise로 sample을 생성합니다. 
2. discriminative model:multilayer perceptron 형태입니다. 
3. 두 model은 backpropagation, dropout 알고리즘 만으로 학습합니다.   
4. generative model로 생성된 sample은 forward propagation에서만 사용됩니다. 

## Related Work

**deep generative models**       
likelihood를 최대화 하는 것이 목적입니다.   
deep Boltzmann machine이 이에 효과적이지만, 계산이 매우 복잡하다는 단점이 있습니다.   
따라서 효과적인 generative machine의 필요성이 대두되었습니다.     
ex) Generative Stochastic Networks: Markov Chain 사용하지 않아도 계산이 가능합니다. 


**Stochastic Backpropagation**
- generator와의 conditional variance를 계산할 수 있으며, 이는 hyperparameter로 사용됩니다. 
- VAEs (Variational Autoencoders)를 학습하는 데에 사용됩니다. 

.


cf) **VAEs**
- 입력 데이터 인코딩에 제약 조건을 추가하는 생성 알고리즘입니다. 
- 잠재된 차원이 정규화 됩니다.
- Variational autoencoder는 데이터를 압축하는 autoencoder와 데이터를 생성하는 GAN,     
  두 가지 역할을 수행할 수 있습니다.


- Second Neural Network와 짝을 이루어 학습됩니다. 
- 인지모델(recognition): 대략적인 추론을 합니다. 

|                            GAN                            |                                 VAEs                                 |
|:---------------------------------------------------------:|:-------------------------------------------------------------------:|
| require Visible units <br> -> cannot model <br> discrete data | require hidden units<br> -> cannot have <br> discrete latent variables |

**NCE** (Noise-Contrastive Estimation) 
- training a generative model by learning the weights     
  that make the model useful for discriminiating data from a fixed noise distribution. 
- "discriminator"가 noise dist와 model dist의 확률 밀도의 ratio로 정해진다는 한계점이 있습니다.   
  -> backpropagation을 두 개의 밀도에 모두 적용해 평가해야 한다는 어려움이 있습니다. 

**Predictability minimization**       
- 두 개의 Neural Network가 compete 하며 성능을 향상시키는 알고리즘입니다. 
- 각각의 hidden unit이 second network의 output으로 학습됩니다. 

|         .                 |                                      Predictability Minimization                                      |                                                                     GAN                                                                    |
|:------------------------:|:--------------------------------------------------------------------------------------------------------:|:------------------------------------------------------------------------------------------------------------------------------------------:|
|        Competition       |                                            only a regularizer                                            |                                            sole training criterion<br>핵심 기준으로 이용됩니다.                                            |
| Nature of<br>Competition | 하나의 Network는 <br>비슷한 output을 생성하려 하며,<br>다른 하나의 Network는 <br>다른 output을 생성하려 합니다.  | 하나의 Network에서 생성된 결과물은 <br>다른 Network의 input으로 사용되며,<br>다른 Network는 그 input이 <br>어떻게 생성되었는지 모르는 상태입니다.  |
|    Learning<br>Process   |                                              목적함수 최소화                                             |   **minimax game**<br>one agent는 최소화, 다른 agent는 최대화를 목표로 합니다. <br>이를 동시에 만족하는 saddle point(안장점)을 찾습니다.   |

## Adversarial nets

$G(z ; \theta_g)$   
- $p_g$ : generator's distribution 
- $p_z(z)$ : input noise variables
- G: parameter $\theta_g$를 가진, 미분 가능한 multilayer perceptron   



$D(x; \theta_d)$   
- multilayer perceptron    
- output: single scalar 
- D: x가 $p_g$로 부터가 아닌, data로 부터 나올 확률 



. 



**D(x) training**
- training examples와 G로부터 온 samples에 옳은 label을 지정할 확률을 최대화 하고자 하며,  
- 동시에 $log(1-D(G(z)))$를 최소화 하고자 합니다. 


.


D와 G는 **two-player minimax game**을 수행합니다. 

![gan5](https://user-images.githubusercontent.com/43749571/78065526-639e5700-73ce-11ea-83c6-aaf414cec7cf.JPG)

< **V(D,G)**가 **최대**가 되도록 학습하는 것은 **판별자(D)**, **최소**화되도록 학습하는 것은 **생성자(G)** >    



**max D**      
- discriminate model이 구별을 잘 한다면, D(x)=1, D(G(x))=0 이 될 것입니다.     
- 즉, **log(D(x)) = 0, log(1-D(G(x))) = $\infty$**    
- 따라서 위 식을 maximize 해야 D가 잘 학습됩니다.     



**min G** 
- generative model이 잘 생성한다면, discriminate는 G(z)를 잘 구분하지 못하게 되며,      
  D(G(z)) = 1이 됩니다.   
- 즉, **log(D(z)) = 1, log(1-D(G(z))) = $-\infty$**
- 따라서 위 식을 minimize 해야 G가 잘 학습됩니다.   

**Optimizing D** 
- inner loop 안에서 D를 계속 optimize 하게 되면 overfitting이 일어날 수 있습니다. 
- 따라서 D를 k-step만큼 optimizing 할 동안, G는 1 step만큼 optimizing을 합니다. 
- G는 천천히 변화하게 됩니다.   

![gan3](https://user-images.githubusercontent.com/43749571/78065565-6f8a1900-73ce-11ea-82bb-a509b19510ad.JPG)

학습 초기에 G가 많이 없을 때, D는 높은 확률로 sample을 기각하게 됩니다.    
이 때, log(1-D(G(z))) 값이 커지게 됩니다.    
G를 training 하여 log(1-D(G(z))) 값을 작게 만들기 보다는,     
log(D(G(z)))값을 크게 하는 방향으로 G의 training을 진행할 수 있습니다.    
이렇게 되면 학습 초기에 목적함수는 강한 gradient를 제공하게 됩니다.   

![gan4](https://user-images.githubusercontent.com/43749571/78065589-79138100-73ce-11ea-87ed-afe890ed79de.JPG)

* blue line: D (discriminate dist)         
* green line: G (generative dist)        
* black line: data generating dist    
* x = G(z) (생성기 G가 noise z를 data space의 x로 mapping)

(a) **real과 fake의 분포가 서로 많이 다를 때**: 
- D가 부분적으로만 정확한 classifier일 때     


(b) **generator를 고정하고 discriminator를 학습시킨 결과**:      
- 알고리즘 D는 sample과 data를 구별하도록 train 되며,     
- $D^*(x) = \frac{p_{data}(x)}{p_{data}(x)+p_g(x)}$ 로 수렴합니다.    


(c) **어느 정도 D의 학습이 이루어지면, G는 D가 구별하기 어려운 방향으로 학습을 하게 됨**:
- G를 업데이트 한 후에, D의 gradient는 G(z)가 data로 분류될 영역으로 유도됩니다.   
- generator는 더 그럴싸한 데이터를 만들어 내게 됩니다.   


(d) **real과 fake가 점점 비슷해짐**:
- 몇 차례의 학습 후에는 $p_g = p_{data}$가 되는 포인트로 도달하게 됩니다.     
- 이 discriminator는 두 distribution을 더 이상 구별하지 못하게 됩니다.    
- 즉, D(x) = 1/2가 됩니다. 

## Theoretical Results

#### 1. Global Optimality of $p_g = p_{data}$

Proposition1.   
G가 고정되어 있을 때, optimal discriminator D는 


$D^*(x) = \frac{p_{data}(x)}{p_{data}(x)+p_g(x)}$

* Global Optimum: $p_g = p_{data}$ 
* $D^*(G) = \frac{1}{2}$
* $C(G)$의 최적값 (Global Minimum) = $-log4$     
  이 때, generative model은 data distribution을 완벽히 복제합니다. 

#### 2. Convergence of Algorithm1 

Proposition2.        
만약 G와 D가 충분한 capacity를 가지고 있고, Algorithm1의 각각의 step에서 discriminator는 주어진 G에 대해 Optimum 값에 도달하려 하며, $p_g$가 다음의 식을 향상시키는 목적으로 update 될 때,  

$\mathbb{E}_{x ~ p_{data}}[logD^*_G(x)] + \mathbb{E}_{x ~ p_g}[log(1 - D^*_G(x))] $ 

$p_g$는 $p_{data}$로 수렴하게 됩니다. 

## Experiment

* Generator nets: recifier linear activaton + sigmoid activation 
* Discriminator nets: maxout activation  
* Dropout: discriminator net에 적용 
* noise: generator network의 맨 하위 계층에서의 input 
* $p_g$: Gaussian Parzen window 기반의 log-likelihood 추정 
* $\sigma$: Gaussians, cross-validation으로 생성됨 

## Advantages and Disadvantages

**disadvantages**       
1. There is no explicit representation of $p_g(x)$
2. D must be synchronized well with G during training 
3. the negative chains of a Boltzmann machine must be kept up to date between learning steps. 



**advantages**
1. Markov Chains are never needed 
2. backprop is used to obtain gradients
3. no inference is needed during learning 
4. wide variety of functions can be incorporated into the model 
5. not being updated directly with data but only with gradients flowing through the discriminator 
6. represent very sharp 

## Conclusions

![gan6](https://user-images.githubusercontent.com/43749571/78065640-8a5c8d80-73ce-11ea-9372-389dcaa1c92e.JPG)

### cf) GAN 코드 예제 (Keras) 
출처: https://pathmind.com/kr/wiki/generative-adversarial-network-gan

In [ ]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)

## 참고자료 
1. https://leedakyeong.tistory.com/entry/%EB%85%BC%EB%AC%B8Generative-Adversarial-NetsGAN
2. https://pathmind.com/kr/wiki/generative-adversarial-network-gan
3. https://gd-story.tistory.com/46
4. https://dhhwang89.tistory.com/27
5. https://everyday-deeplearning.tistory.com/entry/%EC%B4%88-%EA%B0%84%EB%8B%A8-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0Generative-Models-GAN
6. https://haawron.tistory.com/8